In this notebook I redo Georges work but with dataaugmentation in the datagenerator. --Janne

In [2]:
#to work on the server
import os
os.environ["CUDA DEVICE ORDER"]="PCI BUS ID"
os.environ["CUDA VISIBLE DEVICES"]="#gpunum"
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [50]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import applications
from keras import optimizers

from keras import backend as K
K.set_image_dim_ordering('th')
import os

In [25]:
PATH='/home/jankoo/Project/data1/'
files= os.listdir(PATH)
print(files)

['test', 'validation', 'bottleneck_features_train.npy', 'bottleneck_features_validation.npy', 'train']


In [38]:
# dimensions of our images.
img_width, img_height = 150, 150
top_model_weights_path = 'bottleneck_fc_model_jk.h5'
train_data_dir = PATH+'train/'
validation_data_dir =PATH+ 'validation/'
test_dir =PATH+ 'test/'
nb_train_samples = 841
nb_validation_samples = 373
epochs = 50
batch_size = 16


In [27]:
def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255,
                                 rotation_range=360,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 fill_mode='nearest',
                                 horizontal_flip=True,
                                 vertical_flip=True)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save('bottleneck_features_train_jk',
        bottleneck_features_train)

    print("done train")
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save('bottleneck_features_validation_jk',
            bottleneck_features_validation)
    print("done valid")

In [28]:
save_bottleneck_features()

/nobackup/titans/jankoo/anaconda3/lib/python3.6/site-packages/keras/applications/vgg16.py:184: UserWarning: You are using the TensorFlow backend, yet you are using the Theano image data format convention (`image_data_format="channels_first"`). For best performance, set `image_data_format="channels_last"` in your Keras config at ~/.keras/keras.json.
  warnings.warn('You are using the TensorFlow backend, yet you '


Found 841 images belonging to 2 classes.
done train
Found 373 images belonging to 2 classes.
done valid


In [39]:
def train_top_model():
    train_data = np.load('bottleneck_features_train_jk.npy')
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2+nb_train_samples%2))

    validation_data = np.load('bottleneck_features_validation_jk.npy')
    validation_labels = np.array([0] * (nb_validation_samples // 2) + 
                                 [1] * (nb_validation_samples // 2+nb_validation_samples%2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])
    print("starting fitting")
    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)
    print("ended fitting and saving")

In [40]:
train_top_model()

starting fitting
Train on 841 samples, validate on 373 samples
Epoch 1/50
841/841 [==============================] - 1s 2ms/step - loss: 0.9621 - acc: 0.5268 - val_loss: 0.6719 - val_acc: 0.5871
Epoch 2/50
841/841 [==============================] - 0s 330us/step - loss: 0.7080 - acc: 0.5493 - val_loss: 0.8470 - val_acc: 0.5013
Epoch 3/50
841/841 [==============================] - 0s 323us/step - loss: 0.7057 - acc: 0.5541 - val_loss: 0.6763 - val_acc: 0.5871
Epoch 4/50
841/841 [==============================] - 0s 372us/step - loss: 0.6942 - acc: 0.5636 - val_loss: 0.6860 - val_acc: 0.5255
Epoch 5/50
841/841 [==============================] - 0s 360us/step - loss: 0.6745 - acc: 0.5945 - val_loss: 0.7270 - val_acc: 0.5282
Epoch 6/50
841/841 [==============================] - 0s 324us/step - loss: 0.6544 - acc: 0.5945 - val_loss: 0.6782 - val_acc: 0.5496
Epoch 7/50
841/841 [==============================] - 0s 327us/step - loss: 0.6556 - acc: 0.6159 - val_loss: 0.6968 - val_acc: 0.5496
E

Here comes the fine tunning part:

In [46]:
weights_path = '../keras/models/vgg16_weights_tf_dim_ordering_tf_kernels.h5'
input_tensor = Input(shape=(3,img_width,img_height))
base_model = applications.VGG16(weights='imagenet',include_top= False,input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
top_model.load_weights(top_model_weights_path)
model = Model(inputs= base_model.input, outputs= top_model(base_model.output))

/nobackup/titans/jankoo/anaconda3/lib/python3.6/site-packages/keras/applications/vgg16.py:184: UserWarning: You are using the TensorFlow backend, yet you are using the Theano image data format convention (`image_data_format="channels_first"`). For best performance, set `image_data_format="channels_last"` in your Keras config at ~/.keras/keras.json.
  warnings.warn('You are using the TensorFlow backend, yet you '


In [47]:
# set the first 15 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False

In [48]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 3, 150, 150)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 150, 150)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 150, 150)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 75, 75)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 75, 75)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 75, 75)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 37, 37)       0         
__________

In [51]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [52]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=360,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [53]:
#make the generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

Found 841 images belonging to 2 classes.
Found 373 images belonging to 2 classes.


In [54]:
# fine-tune the model
model_history=model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

/nobackup/titans/jankoo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
/nobackup/titans/jankoo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=50, validation_data=<keras.pre..., steps_per_epoch=52, validation_steps=373)`
  import sys


Epoch 1/50
53/52 [==============================] - 17s 322ms/step - loss: 0.6038 - acc: 0.7996 - val_loss: 0.6180 - val_acc: 0.6917
Epoch 2/50
53/52 [==============================] - 13s 238ms/step - loss: 0.4675 - acc: 0.8162 - val_loss: 0.6751 - val_acc: 0.6327
Epoch 3/50
53/52 [==============================] - 12s 235ms/step - loss: 0.4466 - acc: 0.8237 - val_loss: 0.7108 - val_acc: 0.6273
Epoch 4/50
53/52 [==============================] - 13s 243ms/step - loss: 0.4543 - acc: 0.8300 - val_loss: 0.6176 - val_acc: 0.6488
Epoch 5/50
53/52 [==============================] - 12s 235ms/step - loss: 0.4317 - acc: 0.8307 - val_loss: 0.6481 - val_acc: 0.6381
Epoch 6/50
53/52 [==============================] - 13s 238ms/step - loss: 0.4302 - acc: 0.8340 - val_loss: 0.7130 - val_acc: 0.6408
Epoch 7/50
53/52 [==============================] - 12s 230ms/step - loss: 0.4266 - acc: 0.8378 - val_loss: 0.6257 - val_acc: 0.6461
Epoch 8/50
53/52 [==============================] - 12s 236ms/step - 

In [55]:
model.save_weights('last_and_finetuned_jk.h5')